In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame 
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
zomato=pd.read_csv("D:\\kaggle projects\\zomato.csv\\zomato.csv",sep=",",header=0,encoding='latin')

In [ ]:
zomato.columns.tolist()

In [ ]:
zomato.dtypes

In [ ]:
((zomato.isnull() | zomato.isna()).sum() * 100 / zomato.index.size).round(2)

In [ ]:
zomato = zomato.drop(["url","address","phone","dish_liked"],axis=1)

In [ ]:
zomato['rate']=zomato['rate'].str.replace("/5","")
zomato['rate']=zomato['rate'].str.replace(" ","")

In [ ]:
zomato['rate'] = zomato['rate'].replace('NEW',np.NaN)
zomato['rate'] = zomato['rate'].replace('-',np.NaN)

In [ ]:
zomato.dropna(subset=['rate'],inplace=True)

In [ ]:
zomato.rename({'approx_cost(for two people)': 'approx_cost_for_2_people',
               'listed_in(type)':'listed_in_type',
               'listed_in(city)':'listed_in_city'
              }, axis=1, inplace=True)
zomato.columns

In [ ]:
zomato['approx_cost_for_2_people']=zomato['approx_cost_for_2_people'].str.replace(",","")

In [ ]:
#dropping missing values from rest type and cuisines since few missing values are present 
zomato.dropna(subset=['rest_type'],inplace=True)
zomato.dropna(subset=['cuisines'],inplace=True)

In [ ]:
#zomato.votes=pd.to_numeric(zomato.votes)
zomato.approx_cost_for_2_people=pd.to_numeric(zomato.approx_cost_for_2_people)

In [ ]:
zomato['approx_cost_for_2_people'].describe()


In [ ]:
zomato['approx_cost_for_2_people'].fillna(500,inplace=True)# 50% of the time amount value below 500

In [ ]:
zomato.isnull().sum()

In [ ]:
#Encoding
def Encode(zomato):
    for column in zomato.columns[~zomato.columns.isin(['rate', 'approx_cost_for_2_people', 'votes'])]:
        zomato[column] = zomato[column].factorize()[0]
    return zomato

zomato_en = Encode(zomato.copy())

In [ ]:
corr= zomato_en.corr(method='kendall')

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(corr, annot=True)

In [ ]:
x=zomato_en.drop('rate',axis=1)
y=zomato_en['rate']

In [ ]:
import sklearn.model_selection as model_selection
x_train,x_test,y_train,y_test=model_selection.train_test_split(x,y,test_size=0.2,random_state=200)

In [ ]:
print(x_train.shape)
print(y_test.shape)

In [ ]:
#Linear Regression
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(x_train,y_train)

In [ ]:
y_pred=reg.predict(x_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

In [ ]:
#DecisionTreeRegressor

In [ ]:
import sklearn.model_selection as model_selection
x_train,x_test,y_train,y_test=model_selection.train_test_split(x,y,test_size=0.2,random_state=200)

In [ ]:
DTree=DecisionTreeRegressor(min_samples_leaf=3)

In [ ]:
DTree.fit(x_train,y_train)
y_predict=DTree.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test,y_predict)

In [ ]:
#Random Forest Regressor

In [ ]:
RForest=RandomForestRegressor(n_estimators=90,random_state=200,min_samples_leaf=3)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RForest=RandomForestRegressor(n_estimators=50,random_state=1,min_samples_leaf=3)

In [ ]:
RForest.fit(x_train,y_train)
y_predict=RForest.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_predict)

In [ ]:
#With random forest algorithm we are getting better accuraccy than other three

In [ ]:
OR

In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg=RandomForestRegressor(n_estimators=50,max_depth=5,max_features='sqrt',oob_score=True,random_state=1)
reg.fit(x_train,y_train)

In [ ]:
reg.score(x_test,y_test)

In [ ]:
reg.oob_score_

In [ ]:
from sklearn import metrics
metrics.mean_squared_error(y_test,reg.predict(x_test))

In [ ]:
#Tuning the n_estimators
for i in range(50,600,20):
    reg=RandomForestRegressor(n_estimators=i,max_depth=5,max_features='sqrt',oob_score=True,random_state=1)
    reg.fit(x_train,y_train)
    oob=reg.oob_score_
    print('For n_estimators = '+str(i))
    print('OOB score is '+str(oob))
    print('************************')

In [ ]:
reg=RandomForestRegressor(n_estimators=90,max_depth=5,max_features='sqrt',oob_score=True,random_state=1)
reg.fit(x_train,y_train)

In [ ]:
reg.score(x_test,y_test)

In [ ]:
reg.oob_score_

In [ ]:
metrics.mean_squared_error(y_test,reg.predict(x_test))